# Analysis nonA nucleotide use in poly(A) tails

In [1]:
###Load Packages

#Functions for polyA specific analysis (must be in same path)
from polyA_utils import *

#Packages for general column manipulation and statistics
import numpy as np
import pandas as pd
import csv
from tqdm import tqdm
from scipy import stats
import scipy.stats as sci
import pysam

#Packages for plotting
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator
import seaborn as sns
sns.set(font_scale=1.5)
sns.set_style("white")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#Color palette for plots
four_palette = ["darkgreen", "tomato", "steelblue", "orange"]
two_palette = ["steelblue", "orange"]

In [3]:
figpath = "/Users/jacksongordon/Desktop/Papers/Yale_graduate/2023_PABPN1_manuscript/November_2023_edit/plots/"

Question: Is there an increased propensity to use nonA nt's in any of the PABPN1 mutants?

First read in the data

In [4]:
#Function to read polyA tab del files
def read_polyA_reads(file, min_read_count, sample_name, replicate):
    df = pd.read_csv(file, delimiter='\t')
    df = df.loc[df['tail_len'] >= min_read_count]
    df['Sample'] = sample_name
    df['Replicate'] = replicate
    return(df)

In [5]:
#path to the data
path = "/Users/jacksongordon/Desktop/Neugebauer_Lab/Experiments/Exp10_PABPN1_mutants_sequencing/Data_processing/Exp10/"

In [6]:
#Read in dfs, assign to variables
min_A_count = 10

h112_1 = read_polyA_reads(path + 'results/polyA_len/h112.R1_polyA.txt', min_A_count, "EV", "1")
h112_2 = read_polyA_reads(path + 'results/polyA_len/h112.R2_polyA.txt', min_A_count, "EV", "2")

h113_1 = read_polyA_reads(path + 'results/polyA_len/h113.R1_polyA.txt', min_A_count, "WT", "1")
h113_2 = read_polyA_reads(path + 'results/polyA_len/h113.R2_polyA.txt', min_A_count, "WT", "2")

h115_1 = read_polyA_reads(path + 'results/polyA_len/h115.R1_polyA.txt', min_A_count, "4SA", "1")
h115_2 = read_polyA_reads(path + 'results/polyA_len/h115.R2_polyA.txt', min_A_count, "4SA", "2")

h117_1 = read_polyA_reads(path + 'results/polyA_len/h117.R1_polyA.txt', min_A_count, "4SD", "1")
h117_2 = read_polyA_reads(path + 'results/polyA_len/h117.R2_polyA.txt', min_A_count, "4SD", "2")

#Path to gene types df (from gtf)
gene_types = pd.read_csv(path + "config/gene_types_df.tsv", sep = "\t")

#Combine replicates and intersect with gene names
all_reads = pd.concat([h112_1, h112_2, h113_1, h113_2, h115_1, h115_2, h117_1, h117_2], ignore_index = True)
all_reads["gene_id"] = all_reads["gene_id"].str.split(".").str[0]
all_reads = all_reads.merge(gene_types, on = 'gene_id')

Determine whether there is nonA usage in tails. First align the first or last 400 nt of poly(A) tails and calculate the percent of each nucleotide usage for each condition. 

In [ ]:
#Non A bases - run this cell once 
#trange defines the number of bases to consider in the tail alignment
trange = 400
from polyA_utils import *

total_c_h112_1, tail_len_h112_1, base_count_prox_h112_1, base_count_dist_h112_1, base_cov_h112_1 = nonA(path + 'results/mapping/h112.R1_hg38.bam', tail_range = trange)
total_c_h113_1, tail_len_h113_1, base_count_prox_h113_1, base_count_dist_h113_1, base_cov_h113_1 = nonA(path + 'results/mapping/h113.R1_hg38.bam', tail_range = trange)
total_c_h115_1, tail_len_h115_1, base_count_prox_h115_1, base_count_dist_h115_1, base_cov_h115_1 = nonA(path + 'results/mapping/h115.R1_hg38.bam', tail_range = trange)
total_c_h117_1, tail_len_h117_1, base_count_prox_h117_1, base_count_dist_h117_1, base_cov_h117_1 = nonA(path + 'results/mapping/h117.R1_hg38.bam', tail_range = trange)

total_c_h112_2, tail_len_h112_2, base_count_prox_h112_2, base_count_dist_h112_2, base_cov_h112_2 = nonA(path + 'results/mapping/h112.R2_hg38.bam', tail_range = trange)
total_c_h113_2, tail_len_h113_2, base_count_prox_h113_2, base_count_dist_h113_2, base_cov_h113_2 = nonA(path + 'results/mapping/h113.R2_hg38.bam', tail_range = trange)
total_c_h115_2, tail_len_h115_2, base_count_prox_h115_2, base_count_dist_h115_2, base_cov_h115_2 = nonA(path + 'results/mapping/h115.R2_hg38.bam', tail_range = trange)
total_c_h117_2, tail_len_h117_2, base_count_prox_h117_2, base_count_dist_h117_2, base_cov_h117_2 = nonA(path + 'results/mapping/h117.R2_hg38.bam', tail_range = trange)

  0%|                                                                                                                                              | 0/718631 [00:00<?, ?it/s]Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

 63%|████████████████████████████████████████████████████████████████████████████████▊                                               | 76468/121037 [00:30<00:14, 3026.96it/s]

Are there any differences in the percent of each base used between conditions?

In [ ]:
nt_df = pd.concat([
    pd.DataFrame.from_dict(total_c_h112_1, orient = "index").transpose(),
    pd.DataFrame.from_dict(total_c_h112_2, orient = "index").transpose(),

    pd.DataFrame.from_dict(total_c_h113_1, orient = "index").transpose(),
    pd.DataFrame.from_dict(total_c_h113_2, orient = "index").transpose(),

    pd.DataFrame.from_dict(total_c_h115_1, orient = "index").transpose(),
    pd.DataFrame.from_dict(total_c_h115_2, orient = "index").transpose(),

    pd.DataFrame.from_dict(total_c_h117_1, orient = "index").transpose(),
    pd.DataFrame.from_dict(total_c_h117_2, orient = "index").transpose()
        ])

nt_df = nt_df.div(nt_df.sum(axis=1), axis=0)

nt_df["Sample"] = ["EV", "EV", "WT", "WT", "4SA", "4SA", "4SD", "4SD"]
nt_df["Replicate"] = [1, 2, 1, 2, 1, 2, 1, 2]

In [ ]:
nt_df.head()

In [ ]:
f, ax = plt.subplots(1,1, figsize = (5,5))

sns.stripplot(x = nt_df["Sample"], y = nt_df["C"], color = "darkgreen", size = 7)
sns.stripplot(x = nt_df["Sample"], y = nt_df["G"], color = "steelblue", size = 7)
sns.stripplot(x = nt_df["Sample"], y = nt_df["T"], color = "tomato", size = 7)

ax.set_ylabel("percent nt usage")

First create a dataframe of the nonA nt usasge calculated above for ease of plot generation. Then generate plots to look at the percent of each nucleotide used across the alignments

In [ ]:
#Function to create a dataframe from tail composition data
def nonA_df(total_c, tail_len, base_count_prox, base_count_dist, base_cov):
    #EV - 5' alignment
    x=np.arange(1, 401)
    percent_5 = pd.DataFrame()
    percent_5['A'] = base_count_prox[0]/base_cov
    percent_5['G'] = base_count_prox[1]/base_cov
    percent_5['U'] = base_count_prox[2]/base_cov
    percent_5['C'] = base_count_prox[3]/base_cov
    percent_5['reads'] = base_cov
    #EV - 3' alignment
    x=np.arange(1, 401)
    percent_3 = pd.DataFrame()
    percent_3['A'] = base_count_dist[0]/base_cov[::-1]
    percent_3['G'] = base_count_dist[1]/base_cov[::-1]
    percent_3['U'] = base_count_dist[2]/base_cov[::-1]
    percent_3['C'] = base_count_dist[3]/base_cov[::-1]
    percent_3['reads'] = base_cov[::-1]
    return(percent_5, percent_3)

Run the function on all conditions.

In [ ]:
#EV
h112_1_df_5, h112_1_df_3 = nonA_df(total_c_h112_1, tail_len_h112_1, base_count_prox_h112_1, base_count_dist_h112_1, base_cov_h112_1)
h112_2_df_5, h112_2_df_3 = nonA_df(total_c_h112_2, tail_len_h112_2, base_count_prox_h112_2, base_count_dist_h112_2, base_cov_h112_2)
tmp = pd.concat((h112_1_df_5, h112_2_df_5))
h112_percent_5 = tmp.groupby(tmp.index).mean()
tmp = pd.concat((h112_1_df_3, h112_2_df_3))
h112_percent_3 = tmp.groupby(tmp.index).mean()
#WT
h113_1_df_5, h113_1_df_3 = nonA_df(total_c_h113_1, tail_len_h113_1, base_count_prox_h113_1, base_count_dist_h113_1, base_cov_h113_1)
h113_2_df_5, h113_2_df_3 = nonA_df(total_c_h113_2, tail_len_h113_2, base_count_prox_h113_2, base_count_dist_h113_2, base_cov_h113_2)
tmp = pd.concat((h113_1_df_5, h113_2_df_5))
h113_percent_5 = tmp.groupby(tmp.index).mean()
tmp = pd.concat((h113_1_df_3, h113_2_df_3))
h113_percent_3 = tmp.groupby(tmp.index).mean()
#4SA
h115_1_df_5, h115_1_df_3 = nonA_df(total_c_h115_1, tail_len_h115_1, base_count_prox_h115_1, base_count_dist_h115_1, base_cov_h115_1)
h115_2_df_5, h115_2_df_3 = nonA_df(total_c_h115_2, tail_len_h115_2, base_count_prox_h115_2, base_count_dist_h115_2, base_cov_h115_2)
tmp = pd.concat((h115_1_df_5, h115_2_df_5))
h115_percent_5 = tmp.groupby(tmp.index).mean()
tmp = pd.concat((h115_1_df_3, h115_2_df_3))
h115_percent_3 = tmp.groupby(tmp.index).mean()
#4SD
h117_1_df_5, h117_1_df_3 = nonA_df(total_c_h117_1, tail_len_h117_1, base_count_prox_h117_1, base_count_dist_h117_1, base_cov_h117_1)
h117_2_df_5, h117_2_df_3 = nonA_df(total_c_h117_2, tail_len_h117_2, base_count_prox_h117_2, base_count_dist_h117_2, base_cov_h117_2)
tmp = pd.concat((h117_1_df_5, h117_2_df_5))
h117_percent_5 = tmp.groupby(tmp.index).mean()
tmp = pd.concat((h117_1_df_3, h117_2_df_3))
h117_percent_3 = tmp.groupby(tmp.index).mean()

Now plot the percent use of each nucleotide for each condition

In [ ]:
#Plot EV data - all reads
x = np.arange(1, 401)

fig, (ax1, ax3) = plt.subplots(1,2, figsize=(10,5))

ax1 = sns.scatterplot(data = h112_percent_5, x=x, y='G', alpha=.5, s = 50, color = 'darkgreen', ax = ax1)
ax1 = sns.scatterplot(data = h112_percent_5, x=x, y='U', alpha=.5, s = 50, color = 'tomato', ax = ax1)
ax1 = sns.scatterplot(data = h112_percent_5, x=x, y='C', alpha=.5, s = 50, color = 'steelblue', ax = ax1)
ax1.set_xlim([-1, 400])
ax1.set_ylim([0, 0.003])
#ax1.set_yscale('log')
ax1.set_ylabel('fraction')
ax1.legend(['G', 'U', 'C'])
ax1.set_title("5' aligned")
ax3.set_title("3' aligned")
#ax.set_title("S->D 5' tail")
ax2 = ax1.twinx()
ax2 = sns.lineplot(data = h112_percent_5, x=x, y='reads', alpha=.5, color = 'grey', lw = 3)

ax3 = sns.scatterplot(data = h112_percent_3, x=x, y='G', alpha=.5, s = 50, color = 'darkgreen', ax = ax3)
ax3 = sns.scatterplot(data = h112_percent_3, x=x, y='U', alpha=.5, s = 50, color = 'tomato', ax = ax3)
ax3 = sns.scatterplot(data = h112_percent_3, x=x, y='C', alpha=.5, s = 50, color = 'steelblue', ax = ax3)
ax3.set_xlim([0, 400])
ax3.set_ylim([0, 0.003])
#ax1.set_yscale('log')
ax3.set_ylabel('fraction')
ax3.legend(['G', 'U', 'C'])
#ax.set_title("S->D 5' tail")
ax4 = ax3.twinx()
ax4 = sns.lineplot(data = h112_percent_3, x=x, y='reads', alpha=.5, color = 'grey', lw = 3)
ax1.set_xlabel("nucleotide position")
ax3.set_xlabel("nucleotide position")

fig.tight_layout()

In [ ]:
#Plot WT data - all reads
x = np.arange(1, 401)

fig, (ax1, ax3) = plt.subplots(1,2, figsize=(10,5))

ax1 = sns.scatterplot(data = h113_percent_5, x=x, y='G', alpha=.5, s = 50, color = 'darkgreen', ax = ax1)
ax1 = sns.scatterplot(data = h113_percent_5, x=x, y='U', alpha=.5, s = 50, color = 'tomato', ax = ax1)
ax1 = sns.scatterplot(data = h113_percent_5, x=x, y='C', alpha=.5, s = 50, color = 'steelblue', ax = ax1)
ax1.set_xlim([-1, 400])
ax1.set_ylim([0, 0.003])
#ax1.set_yscale('log')
ax1.set_ylabel('fraction')
ax1.legend(['G', 'U', 'C'])
ax1.set_title("5' aligned")
ax3.set_title("3' aligned")
#ax.set_title("S->D 5' tail")
ax2 = ax1.twinx()
ax2 = sns.lineplot(data = h113_percent_5, x=x, y='reads', alpha=.5, color = 'grey', lw = 3)

ax3 = sns.scatterplot(data = h113_percent_3, x=x, y='G', alpha=.5, s = 50, color = 'darkgreen', ax = ax3)
ax3 = sns.scatterplot(data = h113_percent_3, x=x, y='U', alpha=.5, s = 50, color = 'tomato', ax = ax3)
ax3 = sns.scatterplot(data = h113_percent_3, x=x, y='C', alpha=.5, s = 50, color = 'steelblue', ax = ax3)
ax3.set_xlim([0, 400])
ax3.set_ylim([0, 0.003])
#ax1.set_yscale('log')
ax3.set_ylabel('fraction')
ax3.legend(['G', 'U', 'C'])
#ax.set_title("S->D 5' tail")
ax4 = ax3.twinx()
ax4 = sns.lineplot(data = h113_percent_3, x=x, y='reads', alpha=.5, color = 'grey', lw = 3)
ax1.set_xlabel("nucleotide position")
ax3.set_xlabel("nucleotide position")

fig.tight_layout()

In [ ]:
#Plot 4SA data - all reads
x = np.arange(1, 401)

fig, (ax1, ax3) = plt.subplots(1,2, figsize=(10,5))

ax1 = sns.scatterplot(data = h115_percent_5, x=x, y='G', alpha=.5, s = 50, color = 'darkgreen', ax = ax1)
ax1 = sns.scatterplot(data = h115_percent_5, x=x, y='U', alpha=.5, s = 50, color = 'tomato', ax = ax1)
ax1 = sns.scatterplot(data = h115_percent_5, x=x, y='C', alpha=.5, s = 50, color = 'steelblue', ax = ax1)
ax1.set_xlim([-1, 400])
ax1.set_ylim([0, 0.003])
#ax1.set_yscale('log')
ax1.set_ylabel('fraction')
ax1.legend(['G', 'U', 'C'])
ax1.set_title("5' aligned")
ax3.set_title("3' aligned")
#ax.set_title("S->D 5' tail")
ax2 = ax1.twinx()
ax2 = sns.lineplot(data = h115_percent_5, x=x, y='reads', alpha=.5, color = 'grey', lw = 3)

ax3 = sns.scatterplot(data = h115_percent_3, x=x, y='G', alpha=.5, s = 50, color = 'darkgreen', ax = ax3)
ax3 = sns.scatterplot(data = h115_percent_3, x=x, y='U', alpha=.5, s = 50, color = 'tomato', ax = ax3)
ax3 = sns.scatterplot(data = h115_percent_3, x=x, y='C', alpha=.5, s = 50, color = 'steelblue', ax = ax3)
ax3.set_xlim([0, 400])
ax3.set_ylim([0, 0.003])
#ax1.set_yscale('log')
ax3.set_ylabel('fraction')
ax3.legend(['G', 'U', 'C'])
#ax.set_title("S->D 5' tail")
ax4 = ax3.twinx()
ax4 = sns.lineplot(data = h115_percent_3, x=x, y='reads', alpha=.5, color = 'grey', lw = 3)
ax1.set_xlabel("nucleotide position")
ax3.set_xlabel("nucleotide position")

fig.tight_layout()

In [ ]:
#Plot 4SD data - all reads
x = np.arange(1, 401)

fig, (ax1, ax3) = plt.subplots(1,2, figsize=(10,5))

ax1 = sns.scatterplot(data = h117_percent_5, x=x, y='G', alpha=.5, s = 50, color = 'darkgreen', ax = ax1)
ax1 = sns.scatterplot(data = h117_percent_5, x=x, y='U', alpha=.5, s = 50, color = 'tomato', ax = ax1)
ax1 = sns.scatterplot(data = h117_percent_5, x=x, y='C', alpha=.5, s = 50, color = 'steelblue', ax = ax1)
ax1.set_xlim([-1, 400])
ax1.set_ylim([0, 0.003])
#ax1.set_yscale('log')
ax1.set_ylabel('fraction')
ax1.legend(['G', 'U', 'C'])
ax1.set_title("5' aligned")
ax3.set_title("3' aligned")
#ax.set_title("S->D 5' tail")
ax2 = ax1.twinx()
ax2 = sns.lineplot(data = h117_percent_5, x=x, y='reads', alpha=.5, color = 'grey', lw = 3)

ax3 = sns.scatterplot(data = h117_percent_3, x=x, y='G', alpha=.5, s = 50, color = 'darkgreen', ax = ax3)
ax3 = sns.scatterplot(data = h117_percent_3, x=x, y='U', alpha=.5, s = 50, color = 'tomato', ax = ax3)
ax3 = sns.scatterplot(data = h117_percent_3, x=x, y='C', alpha=.5, s = 50, color = 'steelblue', ax = ax3)
ax3.set_xlim([0, 400])
ax3.set_ylim([0, 0.003])
#ax1.set_yscale('log')
ax3.set_ylabel('fraction')
ax3.legend(['G', 'U', 'C'])
#ax.set_title("S->D 5' tail")
ax4 = ax3.twinx()
ax4 = sns.lineplot(data = h117_percent_3, x=x, y='reads', alpha=.5, color = 'grey', lw = 3)
ax1.set_xlabel("nucleotide position")
ax3.set_xlabel("nucleotide position")

fig.tight_layout()

The alignments will be noisly looking at position-to-position differences. Define a function to create bins in which to look at differences. 

In [ ]:
def bin_array(mut, cov, binsize, **kwargs):
    # reactivity calculation in discrete bins of polymerase positions
    assert len(mut) == len(cov)

    bins = np.arange(0, len(mut), binsize)
    mut_bin = np.zeros(len(bins))
    cov_bin = np.zeros(len(bins))

    for i in range(len(bins)):
        if i < len(bins)-1:
            cov_bin[i] = np.sum(cov[bins[i]:bins[i+1]])
            mut_bin[i] = np.sum(mut[bins[i]:bins[i+1]])

        else: # last bin in case it doesn't add up
            cov_bin[i] = np.sum(cov[bins[i]:])
            mut_bin[i] = np.sum(mut[bins[i]:])
            
    return(bins, mut_bin, cov_bin)

In [ ]:
#EV
bins = 5
base_count_prox_a_h112_1_bins = bin_array(base_count_prox_h112_1[0], base_cov_h112_1, bins)
base_count_prox_a_h112_2_bins = bin_array(base_count_prox_h112_2[0], base_cov_h112_2, bins)

base_count_prox_g_h112_1_bins = bin_array(base_count_prox_h112_1[1], base_cov_h112_1, bins)
base_count_prox_g_h112_2_bins = bin_array(base_count_prox_h112_2[1], base_cov_h112_2, bins)

base_count_prox_u_h112_1_bins = bin_array(base_count_prox_h112_1[2], base_cov_h112_1, bins)
base_count_prox_u_h112_2_bins = bin_array(base_count_prox_h112_2[2], base_cov_h112_2, bins)

base_count_prox_c_h112_1_bins = bin_array(base_count_prox_h112_1[3], base_cov_h112_1, bins)
base_count_prox_c_h112_2_bins = bin_array(base_count_prox_h112_2[3], base_cov_h112_2, bins)

base_count_dist_a_h112_1_bins = bin_array(base_count_dist_h112_1[0], base_cov_h112_1, bins)
base_count_dist_a_h112_2_bins = bin_array(base_count_dist_h112_2[0], base_cov_h112_2, bins)

base_count_dist_g_h112_1_bins = bin_array(base_count_dist_h112_1[1], base_cov_h112_1, bins)
base_count_dist_g_h112_2_bins = bin_array(base_count_dist_h112_2[1], base_cov_h112_2, bins)

base_count_dist_u_h112_1_bins = bin_array(base_count_dist_h112_1[2], base_cov_h112_1, bins)
base_count_dist_u_h112_2_bins = bin_array(base_count_dist_h112_2[2], base_cov_h112_2, bins)

base_count_dist_c_h112_1_bins = bin_array(base_count_dist_h112_1[3], base_cov_h112_1, bins)
base_count_dist_c_h112_2_bins = bin_array(base_count_dist_h112_2[3], base_cov_h112_2, bins)

In [ ]:
#WT
bins = 5
base_count_prox_a_h113_1_bins = bin_array(base_count_prox_h113_1[0], base_cov_h113_1, bins)
base_count_prox_a_h113_2_bins = bin_array(base_count_prox_h113_2[0], base_cov_h113_2, bins)

base_count_prox_g_h113_1_bins = bin_array(base_count_prox_h113_1[1], base_cov_h113_1, bins)
base_count_prox_g_h113_2_bins = bin_array(base_count_prox_h113_2[1], base_cov_h113_2, bins)

base_count_prox_u_h113_1_bins = bin_array(base_count_prox_h113_1[2], base_cov_h113_1, bins)
base_count_prox_u_h113_2_bins = bin_array(base_count_prox_h113_2[2], base_cov_h113_2, bins)

base_count_prox_c_h113_1_bins = bin_array(base_count_prox_h113_1[3], base_cov_h113_1, bins)
base_count_prox_c_h113_2_bins = bin_array(base_count_prox_h113_2[3], base_cov_h113_2, bins)

base_count_dist_a_h113_1_bins = bin_array(base_count_dist_h113_1[0], base_cov_h113_1, bins)
base_count_dist_a_h113_2_bins = bin_array(base_count_dist_h113_2[0], base_cov_h113_2, bins)

base_count_dist_g_h113_1_bins = bin_array(base_count_dist_h113_1[1], base_cov_h113_1, bins)
base_count_dist_g_h113_2_bins = bin_array(base_count_dist_h113_2[1], base_cov_h113_2, bins)

base_count_dist_u_h113_1_bins = bin_array(base_count_dist_h113_1[2], base_cov_h113_1, bins)
base_count_dist_u_h113_2_bins = bin_array(base_count_dist_h113_2[2], base_cov_h113_2, bins)

base_count_dist_c_h113_1_bins = bin_array(base_count_dist_h113_1[3], base_cov_h113_1, bins)
base_count_dist_c_h113_2_bins = bin_array(base_count_dist_h113_2[3], base_cov_h113_2, bins)


In [ ]:
#4SA
bins = 5
base_count_prox_a_h115_1_bins = bin_array(base_count_prox_h115_1[0], base_cov_h115_1, bins)
base_count_prox_a_h115_2_bins = bin_array(base_count_prox_h115_2[0], base_cov_h115_2, bins)

base_count_prox_g_h115_1_bins = bin_array(base_count_prox_h115_1[1], base_cov_h115_1, bins)
base_count_prox_g_h115_2_bins = bin_array(base_count_prox_h115_2[1], base_cov_h115_2, bins)

base_count_prox_u_h115_1_bins = bin_array(base_count_prox_h115_1[2], base_cov_h115_1, bins)
base_count_prox_u_h115_2_bins = bin_array(base_count_prox_h115_2[2], base_cov_h115_2, bins)

base_count_prox_c_h115_1_bins = bin_array(base_count_prox_h115_1[3], base_cov_h115_1, bins)
base_count_prox_c_h115_2_bins = bin_array(base_count_prox_h115_2[3], base_cov_h115_2, bins)

base_count_dist_a_h115_1_bins = bin_array(base_count_dist_h115_1[0], base_cov_h115_1, bins)
base_count_dist_a_h115_2_bins = bin_array(base_count_dist_h115_2[0], base_cov_h115_2, bins)

base_count_dist_g_h115_1_bins = bin_array(base_count_dist_h115_1[1], base_cov_h115_1, bins)
base_count_dist_g_h115_2_bins = bin_array(base_count_dist_h115_2[1], base_cov_h115_2, bins)

base_count_dist_u_h115_1_bins = bin_array(base_count_dist_h115_1[2], base_cov_h115_1, bins)
base_count_dist_u_h115_2_bins = bin_array(base_count_dist_h115_2[2], base_cov_h115_2, bins)

base_count_dist_c_h115_1_bins = bin_array(base_count_dist_h115_1[3], base_cov_h115_1, bins)
base_count_dist_c_h115_2_bins = bin_array(base_count_dist_h115_2[3], base_cov_h115_2, bins)


In [ ]:
#4SD
bins = 5
base_count_prox_a_h117_1_bins = bin_array(base_count_prox_h117_1[0], base_cov_h117_1, bins)
base_count_prox_a_h117_2_bins = bin_array(base_count_prox_h117_2[0], base_cov_h117_2, bins)

base_count_prox_g_h117_1_bins = bin_array(base_count_prox_h117_1[1], base_cov_h117_1, bins)
base_count_prox_g_h117_2_bins = bin_array(base_count_prox_h117_2[1], base_cov_h117_2, bins)

base_count_prox_u_h117_1_bins = bin_array(base_count_prox_h117_1[2], base_cov_h117_1, bins)
base_count_prox_u_h117_2_bins = bin_array(base_count_prox_h117_2[2], base_cov_h117_2, bins)

base_count_prox_c_h117_1_bins = bin_array(base_count_prox_h117_1[3], base_cov_h117_1, bins)
base_count_prox_c_h117_2_bins = bin_array(base_count_prox_h117_2[3], base_cov_h117_2, bins)

base_count_dist_a_h117_1_bins = bin_array(base_count_dist_h117_1[0], base_cov_h117_1, bins)
base_count_dist_a_h117_2_bins = bin_array(base_count_dist_h117_2[0], base_cov_h117_2, bins)

base_count_dist_g_h117_1_bins = bin_array(base_count_dist_h117_1[1], base_cov_h117_1, bins)
base_count_dist_g_h117_2_bins = bin_array(base_count_dist_h117_2[1], base_cov_h117_2, bins)

base_count_dist_u_h117_1_bins = bin_array(base_count_dist_h117_1[2], base_cov_h117_1, bins)
base_count_dist_u_h117_2_bins = bin_array(base_count_dist_h117_2[2], base_cov_h117_2, bins)

base_count_dist_c_h117_1_bins = bin_array(base_count_dist_h117_1[3], base_cov_h117_1, bins)
base_count_dist_c_h117_2_bins = bin_array(base_count_dist_h117_2[3], base_cov_h117_2, bins)


In [ ]:
(base_count_dist_u_h115_1_bins[1]/(base_count_dist_u_h115_1_bins[2] + base_count_dist_u_h115_1_bins[1]) + 
base_count_dist_u_h115_2_bins[1]/(base_count_dist_u_h115_2_bins[2] + base_count_dist_u_h115_2_bins[1]))/2

#### Generate figures to compare nonA use between two conditions

C usage - stats comparing 4SA to 4SD

In [ ]:
#Run t-test between percent nt ussage at each postion
stat_vals_5 = stats.ttest_ind(
                            [base_count_dist_c_h115_1_bins[1]/(base_count_dist_c_h115_1_bins[2] + base_count_dist_c_h115_1_bins[1]), base_count_dist_c_h115_2_bins[1]/(base_count_dist_c_h115_2_bins[2] + base_count_dist_c_h115_2_bins[1])],
                            [base_count_dist_c_h117_1_bins[1]/(base_count_dist_c_h117_1_bins[2] + base_count_dist_c_h117_1_bins[1]), base_count_dist_c_h117_2_bins[1]/(base_count_dist_c_h117_2_bins[2] + (base_count_dist_c_h117_2_bins[1]))])[-1]

#calculate FRD
stat_vals_corr_5 = stats.false_discovery_control(stat_vals_5)

#Run t-test between percent nt ussage at each postion
stat_vals_3 = stats.ttest_ind(
                            [base_count_prox_c_h115_1_bins[1]/(base_count_prox_c_h115_1_bins[2] + base_count_prox_c_h115_1_bins[1]), base_count_prox_c_h115_2_bins[1]/(base_count_prox_c_h115_2_bins[2] + base_count_prox_c_h115_2_bins[1])],
                            [base_count_prox_c_h117_1_bins[1]/(base_count_prox_c_h117_1_bins[2] + base_count_prox_c_h117_1_bins[1]), base_count_prox_c_h117_2_bins[1]/(base_count_prox_c_h117_2_bins[2] + (base_count_prox_c_h117_2_bins[1]))])[-1]

#calculate FRD
stat_vals_corr_3 = stats.false_discovery_control(stat_vals_3)

#############################################################

#Figure for percent nt - 5' end A
#Generate plot
fig, (ax2, axs) = plt.subplots(2, 2, gridspec_kw={'height_ratios': [1, 3]}, figsize=(10, 5))

#distal
sns.lineplot(x = (base_count_dist_c_h112_1_bins[0] + base_count_dist_c_h112_2_bins[0])/2, 
             y=(base_count_dist_c_h112_1_bins[1]/(base_count_dist_c_h112_1_bins[2] + base_count_dist_c_h112_1_bins[1])) + 
               (base_count_dist_c_h112_2_bins[1]/(base_count_dist_c_h112_2_bins[2] + base_count_dist_c_h112_2_bins[1]))/2, 
                alpha=.6, color = 'darkgreen', ax = axs[1], linewidth = 5)


sns.lineplot(x = (base_count_dist_c_h113_1_bins[0] + base_count_dist_c_h113_2_bins[0])/2, 
             y=(base_count_dist_c_h113_1_bins[1]/(base_count_dist_c_h113_1_bins[2] + base_count_dist_c_h113_1_bins[1])) + 
               (base_count_dist_c_h113_2_bins[1]/(base_count_dist_c_h113_2_bins[2] + base_count_dist_c_h113_2_bins[1]))/2, 
                alpha=.6, color = 'tomato', ax = axs[1], linewidth = 5)

sns.lineplot(x = (base_count_dist_c_h115_1_bins[0] + base_count_dist_c_h115_2_bins[0])/2, 
             y=(base_count_dist_c_h115_1_bins[1]/(base_count_dist_c_h115_1_bins[2] + base_count_dist_c_h115_1_bins[1])) + 
               (base_count_dist_c_h115_2_bins[1]/(base_count_dist_c_h115_2_bins[2] + base_count_dist_c_h115_2_bins[1]))/2, 
                alpha=.6, color = 'steelblue', ax = axs[1], linewidth = 5)

sns.lineplot(x = (base_count_dist_c_h117_1_bins[0] + base_count_dist_c_h117_2_bins[0])/2, 
             y=(base_count_dist_c_h117_1_bins[1]/(base_count_dist_c_h117_1_bins[2] + base_count_dist_c_h117_1_bins[1])) + 
               (base_count_dist_c_h117_2_bins[1]/(base_count_dist_c_h117_2_bins[2] + base_count_dist_c_h117_2_bins[1]))/2, 
                alpha=.6, color = 'orange', ax = axs[1], linewidth = 5)


ax2[1] = sns.lineplot(x = (base_count_dist_c_h115_1_bins[0] + base_count_dist_c_h115_2_bins[0])/2, 
                   y = -np.log10(stat_vals_5), 
                   alpha=.8, color = 'tomato', ax = ax2[1], linewidth = 5)

ax2[1].set_ylabel("-log10(p)")
ax2[1].axhline(-np.log10(0.05), linestyle = '--', color = 'black')

axs[1].set_xlim(10, 390)
ax2[1].set_xlim(10, 390)
ax2[1].set(xticklabels=[])  # remove the tick labels
ax2[1].tick_params(bottom=False)  # remove the ticks

axs[1].set_ylim(0, 0.1)
ax2[1].set_ylim(0,3)

axs[1].set_ylabel("percent")
axs[1].set_xlabel("distance from 3' end (nt)")

###############################################
#proximal
sns.lineplot(x = (base_count_prox_c_h112_1_bins[0] + base_count_prox_c_h112_2_bins[0])/2, 
             y=(base_count_prox_c_h112_1_bins[1]/(base_count_prox_c_h112_1_bins[2] + base_count_prox_c_h112_1_bins[1])) + 
               (base_count_prox_c_h112_2_bins[1]/(base_count_prox_c_h112_2_bins[2] + base_count_prox_c_h112_2_bins[1]))/2, 
                alpha=.6, color = 'darkgreen', ax = axs[0], linewidth = 5)


sns.lineplot(x = (base_count_prox_c_h113_1_bins[0] + base_count_prox_c_h113_2_bins[0])/2, 
             y=(base_count_prox_c_h113_1_bins[1]/(base_count_prox_c_h113_1_bins[2] + base_count_prox_c_h113_1_bins[1])) + 
               (base_count_prox_c_h113_2_bins[1]/(base_count_prox_c_h113_2_bins[2] + base_count_prox_c_h113_2_bins[1]))/2, 
                alpha=.6, color = 'tomato', ax = axs[0], linewidth = 5)

sns.lineplot(x = (base_count_prox_c_h115_1_bins[0] + base_count_prox_c_h115_2_bins[0])/2, 
             y=(base_count_prox_c_h115_1_bins[1]/(base_count_prox_c_h115_1_bins[2] + base_count_prox_c_h115_1_bins[1])) + 
               (base_count_prox_c_h115_2_bins[1]/(base_count_prox_c_h115_2_bins[2] + base_count_prox_c_h115_2_bins[1]))/2, 
                alpha=.6, color = 'steelblue', ax = axs[0], linewidth = 5)

sns.lineplot(x = (base_count_prox_c_h117_1_bins[0] + base_count_prox_c_h117_2_bins[0])/2, 
             y=(base_count_prox_c_h117_1_bins[1]/(base_count_prox_c_h117_1_bins[2] + base_count_prox_c_h117_1_bins[1])) + 
               (base_count_prox_c_h117_2_bins[1]/(base_count_prox_c_h117_2_bins[2] + base_count_prox_c_h117_2_bins[1]))/2, 
                alpha=.6, color = 'orange', ax = axs[0], linewidth = 5)


ax2[0] = sns.lineplot(x = (base_count_prox_c_h115_1_bins[0] + base_count_prox_c_h115_2_bins[0])/2, 
                   y = -np.log10(stat_vals_3), 
                   alpha=.8, color = 'tomato', ax = ax2[0], linewidth = 5)

ax2[0].set_ylabel("-log10(p)")
ax2[0].axhline(-np.log10(0.05), linestyle = '--', color = 'black')

axs[0].set_xlim(10, 390)
ax2[0].set_xlim(10, 390)
ax2[0].set(xticklabels=[])  # remove the tick labels
ax2[0].tick_params(bottom=False)  # remove the ticks

axs[0].set_ylim(0, 0.1)
ax2[0].set_ylim(0,3)

axs[0].set_ylabel("percent")
axs[0].set_xlabel("distance from cleavage site")


fig.tight_layout()

plt.savefig(figpath + 'nonA_C.png', dpi=300, bbox_inches="tight")

Compare G usage - stats for 4SA vs 4SD

In [ ]:
#Run t-test between percent nt ussage at each postion
stat_vals_5 = stats.ttest_ind(
                            [base_count_dist_g_h115_1_bins[1]/(base_count_dist_g_h115_1_bins[2] + base_count_dist_g_h115_1_bins[1]), base_count_dist_g_h115_2_bins[1]/(base_count_dist_g_h115_2_bins[2] + base_count_dist_g_h115_2_bins[1])],
                            [base_count_dist_g_h117_1_bins[1]/(base_count_dist_g_h117_1_bins[2] + base_count_dist_g_h117_1_bins[1]), base_count_dist_g_h117_2_bins[1]/(base_count_dist_g_h117_2_bins[2] + (base_count_dist_g_h117_2_bins[1]))])[-1]

#calculate FRD
stat_vals_corr_5 = stats.false_discovery_control(stat_vals_5)

#Run t-test between percent nt ussage at each postion
stat_vals_3 = stats.ttest_ind(
                            [base_count_prox_g_h115_1_bins[1]/(base_count_prox_g_h115_1_bins[2] + base_count_prox_g_h115_1_bins[1]), base_count_prox_g_h115_2_bins[1]/(base_count_prox_g_h115_2_bins[2] + base_count_prox_g_h115_2_bins[1])],
                            [base_count_prox_g_h117_1_bins[1]/(base_count_prox_g_h117_1_bins[2] + base_count_prox_g_h117_1_bins[1]), base_count_prox_g_h117_2_bins[1]/(base_count_prox_g_h117_2_bins[2] + (base_count_prox_g_h117_2_bins[1]))])[-1]

#calculate FRD
stat_vals_corr_3 = stats.false_discovery_control(stat_vals_3)

#############################################################

#Figure for percent nt - 5' end A
#Generate plot
fig, (ax2, axs) = plt.subplots(2, 2, gridspec_kw={'height_ratios': [1, 3]}, figsize=(10, 5))

#distal
sns.lineplot(x = (base_count_dist_g_h112_1_bins[0] + base_count_dist_g_h112_2_bins[0])/2, 
             y=(base_count_dist_g_h112_1_bins[1]/(base_count_dist_g_h112_1_bins[2] + base_count_dist_g_h112_1_bins[1])) + 
               (base_count_dist_g_h112_2_bins[1]/(base_count_dist_g_h112_2_bins[2] + base_count_dist_g_h112_2_bins[1]))/2, 
                alpha=.6, color = 'darkgreen', ax = axs[1], linewidth = 5)


sns.lineplot(x = (base_count_dist_g_h113_1_bins[0] + base_count_dist_g_h113_2_bins[0])/2, 
             y=(base_count_dist_g_h113_1_bins[1]/(base_count_dist_g_h113_1_bins[2] + base_count_dist_g_h113_1_bins[1])) + 
               (base_count_dist_g_h113_2_bins[1]/(base_count_dist_g_h113_2_bins[2] + base_count_dist_g_h113_2_bins[1]))/2, 
                alpha=.6, color = 'tomato', ax = axs[1], linewidth = 5)

sns.lineplot(x = (base_count_dist_g_h115_1_bins[0] + base_count_dist_g_h115_2_bins[0])/2, 
             y=(base_count_dist_g_h115_1_bins[1]/(base_count_dist_g_h115_1_bins[2] + base_count_dist_g_h115_1_bins[1])) + 
               (base_count_dist_g_h115_2_bins[1]/(base_count_dist_g_h115_2_bins[2] + base_count_dist_g_h115_2_bins[1]))/2, 
                alpha=.6, color = 'steelblue', ax = axs[1], linewidth = 5)

sns.lineplot(x = (base_count_dist_g_h117_1_bins[0] + base_count_dist_g_h117_2_bins[0])/2, 
             y=(base_count_dist_g_h117_1_bins[1]/(base_count_dist_g_h117_1_bins[2] + base_count_dist_g_h117_1_bins[1])) + 
               (base_count_dist_g_h117_2_bins[1]/(base_count_dist_g_h117_2_bins[2] + base_count_dist_g_h117_2_bins[1]))/2, 
                alpha=.6, color = 'orange', ax = axs[1], linewidth = 5)


ax2[1] = sns.lineplot(x = (base_count_dist_g_h115_1_bins[0] + base_count_dist_g_h115_2_bins[0])/2, 
                   y = -np.log10(stat_vals_5), 
                   alpha=.8, color = 'tomato', ax = ax2[1], linewidth = 5)

ax2[1].set_ylabel("-log10(p)")
ax2[1].axhline(-np.log10(0.05), linestyle = '--', color = 'black')

axs[1].set_xlim(10, 390)
ax2[1].set_xlim(10, 390)
ax2[1].set(xticklabels=[])  # remove the tick labels
ax2[1].tick_params(bottom=False)  # remove the ticks

axs[1].set_ylim(0, 0.1)
ax2[1].set_ylim(0,3)

axs[1].set_ylabel("percent")
axs[1].set_xlabel("distance from 3' end (nt)")

###############################################
#proximal
sns.lineplot(x = (base_count_prox_g_h112_1_bins[0] + base_count_prox_g_h112_2_bins[0])/2, 
             y=(base_count_prox_g_h112_1_bins[1]/(base_count_prox_g_h112_1_bins[2] + base_count_prox_g_h112_1_bins[1])) + 
               (base_count_prox_g_h112_2_bins[1]/(base_count_prox_g_h112_2_bins[2] + base_count_prox_g_h112_2_bins[1]))/2, 
                alpha=.6, color = 'darkgreen', ax = axs[0], linewidth = 5)


sns.lineplot(x = (base_count_prox_g_h113_1_bins[0] + base_count_prox_g_h113_2_bins[0])/2, 
             y=(base_count_prox_g_h113_1_bins[1]/(base_count_prox_g_h113_1_bins[2] + base_count_prox_g_h113_1_bins[1])) + 
               (base_count_prox_g_h113_2_bins[1]/(base_count_prox_g_h113_2_bins[2] + base_count_prox_g_h113_2_bins[1]))/2, 
                alpha=.6, color = 'tomato', ax = axs[0], linewidth = 5)

sns.lineplot(x = (base_count_prox_g_h115_1_bins[0] + base_count_prox_g_h115_2_bins[0])/2, 
             y=(base_count_prox_g_h115_1_bins[1]/(base_count_prox_g_h115_1_bins[2] + base_count_prox_g_h115_1_bins[1])) + 
               (base_count_prox_g_h115_2_bins[1]/(base_count_prox_g_h115_2_bins[2] + base_count_prox_g_h115_2_bins[1]))/2, 
                alpha=.6, color = 'steelblue', ax = axs[0], linewidth = 5)

sns.lineplot(x = (base_count_prox_g_h117_1_bins[0] + base_count_prox_g_h117_2_bins[0])/2, 
             y=(base_count_prox_g_h117_1_bins[1]/(base_count_prox_g_h117_1_bins[2] + base_count_prox_g_h117_1_bins[1])) + 
               (base_count_prox_g_h117_2_bins[1]/(base_count_prox_g_h117_2_bins[2] + base_count_prox_g_h117_2_bins[1]))/2, 
                alpha=.6, color = 'orange', ax = axs[0], linewidth = 5)


ax2[0] = sns.lineplot(x = (base_count_prox_g_h115_1_bins[0] + base_count_prox_g_h115_2_bins[0])/2, 
                   y = -np.log10(stat_vals_3), 
                   alpha=.8, color = 'tomato', ax = ax2[0], linewidth = 5)

ax2[0].set_ylabel("-log10(p)")
ax2[0].axhline(-np.log10(0.05), linestyle = '--', color = 'black')

axs[0].set_xlim(10, 390)
ax2[0].set_xlim(10, 390)
ax2[0].set(xticklabels=[])  # remove the tick labels
ax2[0].tick_params(bottom=False)  # remove the ticks

axs[0].set_ylim(0, 0.1)
ax2[0].set_ylim(0,3)

axs[0].set_ylabel("percent")
axs[0].set_xlabel("distance from cleavage site")


fig.tight_layout()

plt.savefig(figpath + 'NonA_G.png', dpi=300, bbox_inches="tight")

Compare U usage - stats for 4SA vs 4SD

In [ ]:
#Run t-test between percent nt ussage at each postion
stat_vals_5 = stats.ttest_ind(
                            [base_count_dist_u_h115_1_bins[1]/(base_count_dist_u_h115_1_bins[2] + base_count_dist_u_h115_1_bins[1]), base_count_dist_u_h115_2_bins[1]/(base_count_dist_u_h115_2_bins[2] + base_count_dist_u_h115_2_bins[1])],
                            [base_count_dist_u_h117_1_bins[1]/(base_count_dist_u_h117_1_bins[2] + base_count_dist_u_h117_1_bins[1]), base_count_dist_u_h117_2_bins[1]/(base_count_dist_u_h117_2_bins[2] + (base_count_dist_u_h117_2_bins[1]))])[-1]

#calculate FRD
stat_vals_corr_5 = stats.false_discovery_control(stat_vals_5)

#Run t-test between percent nt ussage at each postion
stat_vals_3 = stats.ttest_ind(
                            [base_count_prox_u_h115_1_bins[1]/(base_count_prox_u_h115_1_bins[2] + base_count_prox_u_h115_1_bins[1]), base_count_prox_u_h115_2_bins[1]/(base_count_prox_u_h115_2_bins[2] + base_count_prox_u_h115_2_bins[1])],
                            [base_count_prox_u_h117_1_bins[1]/(base_count_prox_u_h117_1_bins[2] + base_count_prox_u_h117_1_bins[1]), base_count_prox_u_h117_2_bins[1]/(base_count_prox_u_h117_2_bins[2] + (base_count_prox_u_h117_2_bins[1]))])[-1]

#calculate FRD
stat_vals_corr_3 = stats.false_discovery_control(stat_vals_3)

#############################################################

#Figure for percent nt - 5' end A
#Generate plot
fig, (ax2, axs) = plt.subplots(2, 2, gridspec_kw={'height_ratios': [1, 3]}, figsize=(10, 5))

#distal
sns.lineplot(x = (base_count_dist_u_h112_1_bins[0] + base_count_dist_u_h112_2_bins[0])/2, 
             y=(base_count_dist_u_h112_1_bins[1]/(base_count_dist_u_h112_1_bins[2] + base_count_dist_u_h112_1_bins[1])) + 
               (base_count_dist_u_h112_2_bins[1]/(base_count_dist_u_h112_2_bins[2] + base_count_dist_u_h112_2_bins[1]))/2, 
                alpha=.6, color = 'darkgreen', ax = axs[1], linewidth = 5)


sns.lineplot(x = (base_count_dist_u_h113_1_bins[0] + base_count_dist_u_h113_2_bins[0])/2, 
             y=(base_count_dist_u_h113_1_bins[1]/(base_count_dist_u_h113_1_bins[2] + base_count_dist_u_h113_1_bins[1])) + 
               (base_count_dist_u_h113_2_bins[1]/(base_count_dist_u_h113_2_bins[2] + base_count_dist_u_h113_2_bins[1]))/2, 
                alpha=.6, color = 'tomato', ax = axs[1], linewidth = 5)

sns.lineplot(x = (base_count_dist_u_h115_1_bins[0] + base_count_dist_u_h115_2_bins[0])/2, 
             y=(base_count_dist_u_h115_1_bins[1]/(base_count_dist_u_h115_1_bins[2] + base_count_dist_u_h115_1_bins[1])) + 
               (base_count_dist_u_h115_2_bins[1]/(base_count_dist_u_h115_2_bins[2] + base_count_dist_u_h115_2_bins[1]))/2, 
                alpha=.6, color = 'steelblue', ax = axs[1], linewidth = 5)

sns.lineplot(x = (base_count_dist_u_h117_1_bins[0] + base_count_dist_u_h117_2_bins[0])/2, 
             y=(base_count_dist_u_h117_1_bins[1]/(base_count_dist_u_h117_1_bins[2] + base_count_dist_u_h117_1_bins[1])) + 
               (base_count_dist_u_h117_2_bins[1]/(base_count_dist_u_h117_2_bins[2] + base_count_dist_u_h117_2_bins[1]))/2, 
                alpha=.6, color = 'orange', ax = axs[1], linewidth = 5)


ax2[1] = sns.lineplot(x = (base_count_dist_u_h115_1_bins[0] + base_count_dist_u_h115_2_bins[0])/2, 
                   y = -np.log10(stat_vals_5), 
                   alpha=.8, color = 'tomato', ax = ax2[1], linewidth = 5)

ax2[1].set_ylabel("-log10(p)")
ax2[1].axhline(-np.log10(0.05), linestyle = '--', color = 'black')

axs[1].set_xlim(10, 390)
ax2[1].set_xlim(10, 390)
ax2[1].set(xticklabels=[])  # remove the tick labels
ax2[1].tick_params(bottom=False)  # remove the ticks

axs[1].set_ylim(0, 0.1)
ax2[1].set_ylim(0,3)

axs[1].set_ylabel("percent")
axs[1].set_xlabel("distance from 3' end (nt)")

###############################################
#proximal
sns.lineplot(x = (base_count_prox_u_h112_1_bins[0] + base_count_prox_u_h112_2_bins[0])/2, 
             y=(base_count_prox_u_h112_1_bins[1]/(base_count_prox_u_h112_1_bins[2] + base_count_prox_u_h112_1_bins[1])) + 
               (base_count_prox_u_h112_2_bins[1]/(base_count_prox_u_h112_2_bins[2] + base_count_prox_u_h112_2_bins[1]))/2, 
                alpha=.6, color = 'darkgreen', ax = axs[0], linewidth = 5)


sns.lineplot(x = (base_count_prox_u_h113_1_bins[0] + base_count_prox_u_h113_2_bins[0])/2, 
             y=(base_count_prox_u_h113_1_bins[1]/(base_count_prox_u_h113_1_bins[2] + base_count_prox_u_h113_1_bins[1])) + 
               (base_count_prox_u_h113_2_bins[1]/(base_count_prox_u_h113_2_bins[2] + base_count_prox_u_h113_2_bins[1]))/2, 
                alpha=.6, color = 'tomato', ax = axs[0], linewidth = 5)

sns.lineplot(x = (base_count_prox_u_h115_1_bins[0] + base_count_prox_u_h115_2_bins[0])/2, 
             y=(base_count_prox_u_h115_1_bins[1]/(base_count_prox_u_h115_1_bins[2] + base_count_prox_u_h115_1_bins[1])) + 
               (base_count_prox_u_h115_2_bins[1]/(base_count_prox_u_h115_2_bins[2] + base_count_prox_u_h115_2_bins[1]))/2, 
                alpha=.6, color = 'steelblue', ax = axs[0], linewidth = 5)

sns.lineplot(x = (base_count_prox_u_h117_1_bins[0] + base_count_prox_u_h117_2_bins[0])/2, 
             y=(base_count_prox_u_h117_1_bins[1]/(base_count_prox_u_h117_1_bins[2] + base_count_prox_u_h117_1_bins[1])) + 
               (base_count_prox_u_h117_2_bins[1]/(base_count_prox_u_h117_2_bins[2] + base_count_prox_u_h117_2_bins[1]))/2, 
                alpha=.6, color = 'orange', ax = axs[0], linewidth = 5)


ax2[0] = sns.lineplot(x = (base_count_prox_u_h115_1_bins[0] + base_count_prox_u_h115_2_bins[0])/2, 
                   y = -np.log10(stat_vals_3), 
                   alpha=.8, color = 'tomato', ax = ax2[0], linewidth = 5)

ax2[0].set_ylabel("-log10(p)")
ax2[0].axhline(-np.log10(0.05), linestyle = '--', color = 'black')

axs[0].set_xlim(10, 390)
ax2[0].set_xlim(10, 390)
ax2[0].set(xticklabels=[])  # remove the tick labels
ax2[0].tick_params(bottom=False)  # remove the ticks

axs[0].set_ylim(0, 0.1)
ax2[0].set_ylim(0,3)

axs[0].set_ylabel("percent")
axs[0].set_xlabel("distance from cleavage site")


fig.tight_layout()

plt.savefig(figpath + 'nonA_U.png', dpi=300, bbox_inches="tight")

I need a function to determine which reads have nonA usage either at distal end or in general